In [17]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from module import HandDetector

# Create Data

In [4]:
cap = cv2.VideoCapture(0)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = 30  # Adjust the frame rate as needed

videoWriter = cv2.VideoWriter('Train.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    cv2.imshow('Train', frame)
    videoWriter.write(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

In [6]:
landmarks = ['class']

for val in range(0, 21):
    landmarks += [f'x{val}', f'y{val}']

In [8]:
with open('Train.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [9]:
def export_landmark(file, result, action):
    try:
        keypoints = []
        for i in range(len(result)):
            keypoints.append(result[i][0])
            keypoints.append(result[i][1])
            
        keypoints.insert(0, action)
            
        with open(file, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
        
    except Exception as e:
        pass          

In [10]:
cap = cv2.VideoCapture('Train.avi')
detector = HandDetector()

while True:
    ret, frame = cap.read()

    if not ret:
        break
    
    frame = detector.FindHands(frame)
    lm_list_o = detector.FindPositionOriginal()
    
    k = cv2.waitKey(1)
    
    if k == ord('q'):  # Check if 'q' key is pressed to exit the loop
        break
    elif lm_list_o:
        if k == ord('p'):
            export_landmark('Train.csv', lm_list_o, 'pause')
        elif k == ord('r'):
            export_landmark('Train.csv', lm_list_o, 'rewind')
        elif k == ord('f'):
            export_landmark('Train.csv', lm_list_o, 'forward')
        elif k == ord('u'):
            export_landmark('Train.csv', lm_list_o, 'up')
        elif k == ord('d'):
            export_landmark('Train.csv', lm_list_o, 'down')
    
    cv2.imshow('Train', frame) 

cap.release()
cv2.destroyAllWindows()

In [20]:
df_train = pd.read_csv('datasets/Train.csv')

In [21]:
df_train.sample(5)

,class,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x16,y16,x17,y17,x18,y18,x19,y19,x20,y20
970,rewind,0.532728,0.526267,0.463749,0.486333,0.405214,0.415718,0.358610,0.368840,0.322350,...,0.502525,0.315070,0.547950,0.291446,0.537001,0.216754,0.523229,0.284496,0.526788,0.327831
2294,down,0.442782,0.643783,0.404565,0.738985,0.373083,0.796278,0.341927,0.844105,0.317624,...,0.316275,0.623484,0.323567,0.531029,0.274863,0.514659,0.282468,0.555049,0.306665,0.576540
1489,up,0.416882,0.613698,0.387633,0.475151,0.350675,0.367903,0.324471,0.264177,0.319324,...,0.268148,0.600656,0.238731,0.678063,0.231214,0.658890,0.254756,0.661975,0.278292,0.662993
1515,up,0.429772,0.615277,0.400000,0.478653,0.360826,0.371428,0.333339,0.270056,0.329959,...,0.278207,0.604545,0.253789,0.680552,0.246634,0.664900,0.269024,0.667104,0.291909,0.664118
1366,up,0.737862,0.811987,0.708662,0.651915,0.662806,0.523867,0.635142,0.407757,0.632482,...,0.565707,0.759764,0.582527,0.858929,0.526584,0.827324,0.555108,0.827355,0.581996,0.832565


In [23]:
df_train['class'].value_counts()

class
up         586
rewind     524
down       524
pause      520
forward    508
Name: count, dtype: int64

# Train

In [27]:
y = df_train['class']
X = df_train.drop(['class'], axis=1)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
y_train.value_counts()/len(y_train)

class
up         0.223579
rewind     0.199624
pause      0.196806
down       0.190230
forward    0.189760
Name: count, dtype: float64

In [40]:
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier()

In [63]:
y_pred = rf_classifier.predict(X_test)

array(['up'], dtype=object)

In [66]:
rf_classifier.predict_proba(X_test[:1])[0]

array([0., 0., 0., 0., 1.])

In [42]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}".format(accuracy))

Accuracy: 1.00


In [44]:
y_pred[:10]

array(['up', 'forward', 'down', 'forward', 'rewind', 'down', 'down',
       'down', 'pause', 'down'], dtype=object)

In [47]:
with open('rfclassifier.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)

# Test Performance

In [81]:
import time

In [56]:
with open('rfclassifier.pkl', 'rb') as file:
    model = pickle.load(file)

In [93]:
cap = cv2.VideoCapture(0)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = 30  # Adjust the frame rate as needed

detector = HandDetector()

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    frame = detector.FindHands(frame)
    
    lm_list_o = detector.FindPositionOriginal()
    
    command_status = 'Unknown'  # Default value
    
    if lm_list_o:
        
        keypoints = []
        for i in range(len(lm_list_o)):
            keypoints.append(lm_list_o[i][0])
            keypoints.append(lm_list_o[i][1])
        
        x = np.array(keypoints).reshape(1, -1)
        
        command_class = model.predict(x)[0]
        command_prob = model.predict_proba(x)[0][model.predict_proba(x)[0].argmax()]

        if command_prob >= 0.7:
            command_status = command_class
        else:
            command_status = 'Unknown'
    
        cv2.rectangle(frame, (0,0), (500, 120), (245, 117, 16), -1)
    
        cv2.putText(frame, f"command: {command_status}", (10, 35), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
        cv2.putText(frame, f"probs: {command_prob}", (10, 85), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
        
    cv2.imshow('Train', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

...

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando

/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/bob/opt/anaconda3/envs/air_mouse/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but Rando